<a href="https://colab.research.google.com/github/meghnavarma25/SRIP/blob/main/malariab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2 as cv
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, MaxPool2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import Activation, Convolution2D, Dropout, Conv2D,AveragePooling2D, BatchNormalization,Flatten,GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.preprocessing import image
from sklearn.utils import shuffle
import pandas as pd

images = []
labels  = []
TRAIN_DIR = '/content/drive/MyDrive/images/train'
IMG_SIZE = (240,240) # Image resolution
BATCH_SIZE = 8

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

infected_dir = '/content/drive/MyDrive/images/cell_images/Parasitized'
uninfected_dir = '/content/drive/MyDrive/images/cell_images/Uninfected'


num_infected_images = len(os.listdir(infected_dir))
num_uninfected_images = len(os.listdir(uninfected_dir))

print(f"Number of infected images: {num_infected_images}")
print(f"Number of uninfected images: {num_uninfected_images}")

Number of infected images: 13780
Number of uninfected images: 13780


In [5]:
dataset_path=os.listdir('/content/drive/MyDrive/images/cell_images')
class_labels=[]
for item in dataset_path:
  all_classes=os.listdir('/content/drive/MyDrive/images/cell_images'+'/'+item)
  for room in all_classes:
    class_labels.append((item,str('dataset_path'+'/'+item)+'/'+room))

In [ ]:
df=pd.DataFrame(data=class_labels,columns=['Labels','image'])
print(df.head())
print(df.tail())

        Labels                                              image
0  Parasitized  dataset_path/Parasitized/C52P13thinF_IMG_20150...
1  Parasitized  dataset_path/Parasitized/C174P135NThinF_IMG_20...
2  Parasitized  dataset_path/Parasitized/C189P150ThinF_IMG_201...
3  Parasitized  dataset_path/Parasitized/C51AP12thinF_IMG_2015...
4  Parasitized  dataset_path/Parasitized/C184P145ThinF_IMG_201...
           Labels                                              image
22043  Uninfected  dataset_path/Uninfected/C12NThinF_IMG_20150614...
22044  Uninfected  dataset_path/Uninfected/C235ThinF_IMG_20151112...
22045  Uninfected  dataset_path/Uninfected/C163P124ThinF_IMG_2015...
22046  Uninfected  dataset_path/Uninfected/C72P33_ThinF_IMG_20150...
22047  Uninfected  dataset_path/Uninfected/C85P46ThinF_IMG_201508...


In [ ]:

def copy_images(images, labels, infected_dir, uninfected_dir, dest_dir):
    for img, label in zip(images, labels):
        if label == 'Parasitized':
            src_path = os.path.join(infected_dir, img)
        else:
            src_path = os.path.join(uninfected_dir, img)
        dest_path = os.path.join(dest_dir, label, img)
        shutil.copy(src_path, dest_path)

In [3]:
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import shutil

infected_dir = '/content/drive/MyDrive/images/cell_images/Parasitized'
uninfected_dir = '/content/drive/MyDrive/images/cell_images/Uninfected'

infected_images = os.listdir(infected_dir)
uninfected_images = os.listdir(uninfected_dir)

infected_labels = ['Parasitized'] * len(infected_images)
uninfected_labels = ['Uninfected'] * len(uninfected_images)


all_images = infected_images + uninfected_images
all_labels = infected_labels + uninfected_labels


train_images, test_images, train_labels, test_labels = train_test_split(
    all_images, all_labels, test_size=0.2, stratify=all_labels, random_state=42
)

train_dir = '/content/drive/MyDrive/images/train'
test_dir = '/content/drive/MyDrive/images/test'

train_infected_dir = os.path.join(train_dir, 'Parasitized')
train_uninfected_dir = os.path.join(train_dir, 'Uninfected')
test_infected_dir = os.path.join(test_dir, 'Parasitized')
test_uninfected_dir = os.path.join(test_dir, 'Uninfected')

os.makedirs(train_infected_dir, exist_ok=True)
os.makedirs(train_uninfected_dir, exist_ok=True)
os.makedirs(test_infected_dir, exist_ok=True)
os.makedirs(test_uninfected_dir, exist_ok=True)

#copy_images(train_images, train_labels, infected_dir, uninfected_dir, train_dir)
#copy_images(test_images, test_labels, infected_dir, uninfected_dir, test_dir)

print(f"Training set: {len(train_images)} images")
print(f"Testing set: {len(test_images)} images")


Training set: 22048 images
Testing set: 5512 images


In [ ]:
train_para = '/content/drive/MyDrive/images/train/Parasitized'
train_unpara='/content/drive/MyDrive/images/train/Uninfected'
test_para='/content/drive/MyDrive/images/test/Parasitized'
test_unpara='/content/drive/MyDrive/images/test/Uninfected'

num_trp = len(os.listdir(train_para))
num_trup = len(os.listdir(train_unpara))
num_tsp=len(os.listdir(test_para))
num_tsup=len(os.listdir(test_unpara))

print(f"Number of infected images in train: {num_trp}")
print(f"Number of uninfected images in train: {num_trup}")
print(f"Number of infected images in test: {num_tsp}")
print(f"Number of uninfected images in test: {num_tsup}")

Number of infected images in train: 11024
Number of uninfected images in train: 11024
Number of infected images in test: 2756
Number of uninfected images in test: 2756


In [4]:
import tensorflow as tf

TRAIN_DIR = '/content/drive/MyDrive/images'


BATCH_SIZE = 32  # Adjust as needed


train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    image_size=(240, 240),
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset='training',
    seed=123,
    label_mode='categorical',
)

valid_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,  # Same split ratio
    image_size=(240, 240),
    batch_size=BATCH_SIZE,
    shuffle=False,
    subset='validation',
    seed=123,
    label_mode='categorical',
)

print(f"Number of batches in training data: {len(train_data)}")
print(f"Number of batches in validation data: {len(valid_data)}")

num_train_samples = len(train_data.file_paths)
num_valid_samples = len(valid_data.file_paths)
print(f"Number of training samples: {num_train_samples}")
print(f"Number of validation samples: {num_valid_samples}")



Found 22047 files belonging to 2 classes.
Using 17638 files for training.
Found 22047 files belonging to 2 classes.
Using 4409 files for validation.
Number of batches in training data: 552
Number of batches in validation data: 138
Number of training samples: 17638
Number of validation samples: 4409


In [ ]:
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
height = 240
width = 240
channels = 3
input_shape = (height, width, channels)

In [ ]:
input_shape = (240, 240, 3)

# Preprocessing layers
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(240, 240),
    layers.experimental.preprocessing.Rescaling(1/255)
])

# Data augmentation to reduce overtraining
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=input_shape),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])

# Load the EfficientNetB1 model
efnb1 = EfficientNetB1(weights='imagenet', include_top=False, input_shape=input_shape)

# Build the model
model = Sequential([
    resize_and_rescale,
    data_augmentation,
    efnb1,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(2, activation='softmax')
])
model.build((None, 240, 240, 3))
# Model summary
model.summary()

27018416/27018416 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 240, 240, 3)       0         
                                                                 
 sequential_1 (Sequential)   (None, 240, 240, 3)       0         
                                                                 
 efficientnetb1 (Functional  (None, 8, 8, 1280)        6575239   
 )                                                               
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                       

In [ ]:
optimizer = Adam(learning_rate=0.0001)


early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)
rlrop = ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-6, verbose=1)


model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=5,
    callbacks=[early_stop, rlrop],
    verbose=1
)

Epoch 1/5
2205/2205 [==============================] - 7574s 3s/step - loss: 0.1635 - accuracy: 0.9433 - val_loss: 0.1972 - val_accuracy: 0.9989 - lr: 1.0000e-04
Epoch 2/5
2205/2205 [==============================] - 7528s 3s/step - loss: 0.1155 - accuracy: 0.9628 - val_loss: 0.5643 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 3/5
2205/2205 [==============================] - 7531s 3s/step - loss: 0.1030 - accuracy: 0.9663 - val_loss: 0.2119 - val_accuracy: 1.0000 - lr: 1.0000e-04
Epoch 4/5
2205/2205 [==============================] - 7598s 3s/step - loss: 0.0906 - accuracy: 0.9700 - val_loss: 0.1594 - val_accuracy: 0.9807 - lr: 1.0000e-04
Epoch 5/5
2205/2205 [==============================] - 7544s 3s/step - loss: 0.0837 - accuracy: 0.9705 - val_loss: 0.3411 - val_accuracy: 1.0000 - lr: 1.0000e-04


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(240, 240),#size for efficientnet b1
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


Found 5511 images belonging to 2 classes.
173/173 [==============================] - 1038s 6s/step - loss: 0.8092 - accuracy: 0.4999
Test Loss: 0.8092149496078491
Test Accuracy: 0.49990928173065186


In [ ]:
model.save('/content/drive/MyDrive/saved models/B1_with_100%_val_acc.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
